## Import packages

In [1]:
from big_phoney import BigPhoney
import lyricsgenius as genius
import random
import nlpaug.augmenter.word as naw
import re
import numpy as np

#----------------For future live text generation-----------------
#import gpt_2_simple as gpt2

# model_name = "124M"
# model is saved into current directory under /models/124M/
# gpt2.download_gpt2(model_name=model_name) #need to run only once. comment out once done.

# Load models
#sess = gpt2.start_tf_sess()
#gpt2.load_gpt2(sess, run_name='run1')

#----------------------------------------------------------------

Using TensorFlow backend.


## Functions 

In [2]:
def get_target_lyrics(artist, title, api_key):
    
    api = genius.Genius(api_key)
    api.remove_section_headers = True # Removes section headers like "Verse" and "Intro"
    
    #get song lyrics
    song = api.search_song(title, artist)
    
    #clean up the target lyrics
    target = song.lyrics
    target = target.replace("\u2005"," ")
    target = target.replace("\\"," ")
    target = target.replace("\\n"," ")
    target = target.replace("("," ")
    target = target.replace(")"," ")
    target = target.replace("\n\n","\n")
    target = target.replace("\n\n\n","\n")
    target = target.replace("2x","")
    #create list of lines
    target_lyrics = target.split("\n")
    print("Target lyrics:")
    print(target_lyrics)
    return target_lyrics

In [ ]:
get_target_lyrics('Idina Menzel','Let it go')

In [ ]:
#----------------For future live text generation-----------------
#def generate_lyrics(): 
#    
#    gen_lyrics = gpt2.generate(sess, 
#                     length=250,
#                     temperature=0.9,
#                     include_prefix=False,
#                     #prefix="Deep in the night",
#                     nsamples=1,
#                     batch_size=1,
#                     return_as_list=True)[0]
#    
#    gen_lyrics = gen_lyrics.split("\n")
#    
#    return gen_lyrics
#----------------------------------------------------------------

In [3]:
def lyrics_to_list(file_name):
    with open(file_name, 'r') as file:
        data = file.read().split("====================")
        random_song = random.choice(data)
        target = random_song
        
        # clean generatedd lyrics
        target = target.replace("\u2005"," ")
        target = target.replace("\\"," ")
        target = target.replace("\\n"," ")
        target = target.replace("("," ")
        target = target.replace(")"," ")
        target = target.replace("\n\n","\n")
        target = target.replace("\n\n\n","\n")
        target = target.replace("2x","")
        gen_lyrics = target.split("\n")
        gen_lyrics = gen_lyrics[1:-1]
        print("Gen lyrics:")
        print(gen_lyrics)
    return gen_lyrics

#### The magical lyrics fitting function

This function takes lyrics generated by a fine-tuned GPT-2 model, and fits them to the syllabic structure of target lyrics (matches the number of syllables per line). 

In [34]:
def fit_lyrics(gen_lyrics, target_lyrics):
    
    print(gen_lyrics)
    
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")
    print('aug')
    
    phoney = BigPhoney()
    print('initialized phoney')
    
    # Counts the number of syllables in each line
    def count_syls(text):
    
        schema = []
    
        for line in text:
            syls = phoney.count_syllables(line)
            schema.append(syls)
            #print(syls,line)
        
        return schema
    
    # Remove characters and contractions from the line. This will make it easier to
    # augment lines that need augmentation. 
    def decontracted(phrase):
        # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        phrase = re.sub('[!@#$?]', '', phrase)
        return phrase
    
    # Count the number of syllables in the generated and target texts.
    gen_schema = count_syls(gen_lyrics)
    target_schema = count_syls(target_lyrics)
#    print(target_schema)

    # make generated lyrics same length as target lyrics
    target_len = len(target_schema)
    del gen_schema[target_len:]
    del gen_lyrics[target_len:]
    
    # initialize array for the new fitted lyrics
    new_lyrics = []
    
    # loop through each line and either find existing line to place into the current position, 
    # or augment the current line. 
    
    # keep track of repeated lines
    repeats = np.array([])
    for num, line in enumerate(gen_lyrics):
        print("line in gen_lyrics:")
        print(num)
        print(line)
        ##Right now gen_lyrics changes through each iteration, which is not good
        print("length of gen_lyrics:")
        print(len(gen_lyrics))
        
        # if the line is already the right length, add it to the new lyrics. 
        if (gen_schema[num] == target_schema[num]): 
            new_lyrics.append(line)
            
        # if the line is not the right length, keep going until we find one that is
        elif (gen_schema[num] != target_schema[num]):
            # keep track of the number of syllables in the current line
            syls = gen_schema[num]
            # keep track of which line we're on 
            track = 0
            # while the line is not the target length,
            while syls != target_schema[num]:
                
                if track == len(gen_lyrics)-1:
                    end = True
                    break
                else:
                    # go to the next line
                    track = track + 1
                    # check the num of syls
                    new_line = gen_lyrics[track]
                    syls = phoney.count_syllables(new_line)
                    end = False
            print("syls:")
            print(syls)
            print("target schema syls")
            print(target_schema[num])
            # if we found a line that's the right length before we reached the end,
            # add the line to the new lyrics
            if end == False:
                new_lyrics.append(new_line)
                repeats = np.append(repeats, track)
                if np.count_nonzero(repeats == track) >= 2:
                    print("repeats greater than 3")
                    print(new_line)
                    gen_lyrics.pop(track)
                    gen_schema.pop(track)
                    print("length of gen_lyrics after pop")
                    print(len(gen_lyrics))
                    print("length of gen_schema after pop")
                    print(len(gen_schema))
                
            #If we didn't find a line that's the right length,
            if end:
                line = decontracted(line)
                print("No matching line")
                #print(line)
                syls = gen_schema[num]
                while syls < target_schema[num]:
#                    print("not enough syls")
                    original_line = line
                    line = aug.augment(line)
                    line = re.sub(r'[^\w\s]','',line)
                    print(line)
                    syls = phoney.count_syllables(line)
                #In case we overshoot (add too many syllables)
                    if syls > target_schema[num]:
                        print("Oops we overshot")
                        line = original_line
                        syls = phoney.count_syllables(line)
                new_line = line
#                print("the same line:")
                syls = gen_schema[num]
                words = line.split(" ")
                while syls > target_schema[num]:
                    print("too many syls")
                    original_words = words
                    #instead of deleting the last word, try deleting a word randomly
                    #del words[-1]
                    words.pop(random.randrange(len(words))) 
                    new_line = ' '.join(words)
                    syls = phoney.count_syllables(new_line)
                    #In case too many syllables are deleted
                    if syls < target_schema[num]:
                        print("Oops, deleted too many")
                        words = original_words
                        new_line = ' '.join(words)
                        syls = phoney.count_syllables(new_line)
                        
                new_lyrics.append(new_line)
                
    return new_lyrics


## Time to test everything

In [50]:
artist = "Idina Menzel"
title = "Let it go"

In [51]:
lines = open('API.txt').readlines()
api_key = lines[0].strip()
target_lyrics = get_target_lyrics(artist, title, api_key)
gen_lyrics = lyrics_to_list('gpt2_gentext_slayer.txt')

Searching for "Let it go" by Idina Menzel...
Done.
Target lyrics:
['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, heaven knows I've tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on', 'The cold never bothered me anyway', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', 'Let it go  go, go, go go, go go, go go, go, go, go go ', 'Let it go', 'Let it go', 'Let it go', "It's funny how some distance makes everything seem small", "And the fears that once controlled me can't get to me at all"

In [52]:
new_lyrics = fit_lyrics(gen_lyrics, target_lyrics)

['brazen savage, with no remorse', 'aaaaah!!', 'war!!!', 'the war!!!', 'it was the war!!!', 'it was the war!!!', 'it was the war!!!', 'when ive-grown-out-a-fantasy-now', 'i-am-the-next-gen!!!', 'i-am the next fuckin ive come', 'engrave my hidden cameras', 'take pictures of your pain', 'take a closer look at me', 'look into my eyes', 'feel your pain', 'and see what ive gone wrong', 'youll do anything, anything', 'youll do anything to make me go', 'die in front of so many people', 'youll do anything, anything', 'youll do anything to make me go', 'die in front of so many people', 'haventame the one thats done all the fucking magic', 'enticing the whole is that just me', 'a dick of shit that doesnt have a soul', 'a whore of bullshit that doesnt want one', 'a complete fucking mess that doesnt belong', 'a complete fucking mess that doesnt belong', 'chaos is my reality', 'playing the part of you messiah', 'the one who giveth, get rich quick', 'youve found yourself in this mess', 'you know wha

In [53]:
print(target_lyrics)

['The snow glows white on the mountain tonight', 'Not a footprint to be seen', 'A kingdom of isolation', "And it looks like I'm the queen", 'The wind is howling like this swirling storm inside', "Couldn't keep it in, heaven knows I've tried", "Don't let them in, don't let them see", 'Be the good girl you always have to be', "Conceal, don't feel, don't let them know", 'Well, now they know', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', "I don't care what they're going to say", 'Let the storm rage on', 'The cold never bothered me anyway', 'Let it go, let it go', "Can't hold it back anymore", 'Let it go, let it go', 'Turn away and slam the door', 'Let it go  go, go, go go, go go, go go, go, go, go go ', 'Let it go', 'Let it go', 'Let it go', "It's funny how some distance makes everything seem small", "And the fears that once controlled me can't get to me at all", "It's time to see what I can do", 'To test the limits and break 

In [54]:
print(gen_lyrics)

['brazen savage, with no remorse', 'war!!!', 'the war!!!', 'it was the war!!!', 'it was the war!!!', 'it was the war!!!', 'when ive-grown-out-a-fantasy-now', 'i-am-the-next-gen!!!', 'look into my eyes', 'youll do anything, anything', 'youll do anything to make me go', 'die in front of so many people', 'youll do anything, anything', 'youll do anything to make me go', 'die in front of so many people', 'haventame the one thats done all the fucking magic', 'enticing the whole is that just me', 'a dick of shit that doesnt have a soul', 'a whore of bullshit that doesnt want one', 'a complete fucking mess that doesnt belong', 'a complete fucking mess that doesnt belong', 'chaos is my reality', 'playing the part of you messiah', 'the one who giveth, get rich quick', 'you know what']


In [55]:
print(new_lyrics)

['a dick of shit that doesnt have a soul', 'take a closer look at me', 'i-am the next fuckin ive come', 'take a closer look at me', ' do you ever even even really know what', 'a dick of shit that doesnt have a soul', 'i-am the next fuckin ive come', 'a dick of shit that doesnt have a soul', 'engrave my hidden cameras', 'it was the war!!!', 'take pictures of your pain', 'youve found yourself in this mess', 'and see what ive gone wrong', 'youve found yourself in this mess', 'when ive-grown-out-a-fantasy-now', 'i-am-the-next-gen!!!', 'a dick of shit that doesnt have a soul', 'and see what ive gone wrong', 'you just never did know or', 'you never , did', 'you guys will know by now now', 'and you will probably certainly never even know what', 'aaaaah!!', 'aaaaah!!', 'feel your pain']


In [ ]:
print(target_schema)
print(gen_schema)

In [ ]:
# End tensorflow session 
sess.close()

In [ ]:
lines = open('TEST.txt').readlines()
api_key = lines[0].strip()

In [ ]:
print(api_key)

In [13]:
phoney = BigPhoney()
print('initialized phoney')

initialized phoney


In [63]:
new1 = phoney.count_syllables(new_lyrics[20])
new2 = phoney.count_syllables(new_lyrics[17])
new3 = phoney.count_syllables(new_lyrics[18])
print(new1,new2,new3)

7 6 7


In [64]:
target1 = phoney.count_syllables(target_lyrics[20])
target2 = phoney.count_syllables(target_lyrics[17])
target3 = phoney.count_syllables(target_lyrics[18])
print(target1,target2,target3)

7 6 7


In [43]:
print(target_lyrics[11])
print(new_lyrics[11])

Can't hold it back anymore
religion is violence
